# Import Libraries

In [1]:
import torch
import os
import dill
import torch
import mlflow
import pandas as pd
from torch import nn
from torch.utils.data import DataLoader
from sklearn.preprocessing import OneHotEncoder
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from tqdm.notebook import tqdm
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

# Parameters

In [2]:
epochs        = 150
batch_size    = 32
learning_rate = 0.001


# Read data and pre-processing

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

data = pd.read_csv('StudentsPerformance.csv')
feature_X = data.iloc[:, :5]
target = data['math score']
target = target/target.max()

# Feature engineering

In [5]:
# training test split
X_train, X_test, y_train, y_test = train_test_split(feature_encoded, target, test_size=0.10, random_state=1)

# Separate 10% of training data as validation set
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=1)


ohe = OneHotEncoder()
ohe.fit(feature_X)
feature_encoded = ohe.transform(feature_X)
feature_encoded = pd.DataFrame(feature_encoded.toarray())
feature_size  = len(X_train.columns)

# Training test split

In [6]:
class InputData(Dataset):
    
    def __init__(self, features, target):
        self.features = features
        self.target   = target
        
    def __getitem__(self, index):
        return self.features[index], self.target[index]
        
    def __len__ (self):
        return len(self.features)

train_dataset = InputData(torch.from_numpy(X_train.values).float(), torch.from_numpy(y_train.values).float())
val_dataset   = InputData(torch.from_numpy(X_val.values).float(),   torch.from_numpy(y_val.values).float())
test_dataset  = InputData(torch.from_numpy(X_test.values).float(),  torch.from_numpy(y_test.values).float())


# Data Loader

In [7]:
train_loader = DataLoader(dataset  = train_dataset, batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(dataset  = val_dataset,   batch_size=1)
test_loader  = DataLoader(dataset  = test_dataset,  batch_size=1)

# Model training 

In [8]:
class Regression(nn.Module):
    def __init__(self, num_features):
        super(Regression, self).__init__()
        
        self.layer_1 = nn.Linear(num_features, 16)
        self.layer_2 = nn.Linear(16, 10)
        self.out = nn.Linear(10, 1)
        
        self.relu = nn.ReLU()
        
    def forward(self, inputs):
        x = self.relu(self.layer_1(inputs))
        x = self.relu(self.layer_2(x))
        x = self.out(x)
        return (x)

In [9]:
model = Regression(feature_size)
model.to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


loss_stats = {
    'train': [],
    "val": []}

epochs = 200


from tqdm.notebook import tqdm


print("Training started !!!")
for epoch in range(1, epochs+1):
  
    model.train()
    for X_train_batch, y_train_batch in train_loader:
        X_train_batch, y_train_batch = X_train_batch.to(device), y_train_batch.to(device)
        optimizer.zero_grad()
        
        y_train_pred = model(X_train_batch)
      
        train_loss = criterion(y_train_pred, y_train_batch.unsqueeze(1))
      
        train_loss.backward()
        optimizer.step()
        
    # validation    
    with torch.no_grad():
        
         model.eval()
         for X_val_batch, y_val_batch in val_loader:
             X_val_batch, y_val_batch = X_val_batch.to(device), y_val_batch.to(device)
            
             y_val_pred = model(X_val_batch)
                        
             val_loss = criterion(y_val_pred, y_val_batch.unsqueeze(1))

    valIdation_loss = float(val_loss.detach().numpy())
    training_loss   = float(train_loss.detach().numpy())
    print(valIdation_loss, training_loss)


X, y = next(iter(test_loader))
print(f"Model input: {X.size()}")
torch_out = model(X.to("cpu"))
print(f"Model output: {torch_out.detach().cpu().size()}")


Training started !!!
0.22132039070129395 0.20992720127105713
0.13580422103405 0.11451031267642975
0.05662846937775612 0.09548471868038177
0.002188674174249172 0.021542105823755264
0.002020004903897643 0.027117732912302017
0.0006218467024154961 0.016726108267903328
0.00030033683287911117 0.018831761553883553
0.00017998565454035997 0.02517317607998848
7.913763693068177e-05 0.023975053802132607
3.638206180767156e-06 0.014302466996014118
6.681041122647002e-05 0.0224201250821352
1.0151276001124643e-05 0.008712304756045341
2.9789734981022775e-05 0.02287457324564457
8.351372002834978e-07 0.008252376690506935
6.57448181300424e-05 0.018535520881414413
3.0695049645146355e-05 0.03268510848283768
1.667119431658648e-05 0.034128863364458084
1.0424658285046462e-05 0.024841878563165665
3.14385488309199e-07 0.014801422134041786
0.00011366848775651306 0.016000036150217056
3.293308327556588e-05 0.023629222065210342
1.2333900485828053e-05 0.014173755422234535
3.248806024203077e-08 0.02146102488040924
6.21

0.00014190824003890157 0.01434040255844593
0.00019917702593374997 0.008114405907690525
6.3567073084414e-05 0.0136495903134346
2.7259616786068364e-10 0.015805793926119804
0.000274058518698439 0.011205870658159256
0.00019525298557709903 0.01121935062110424
0.00014875705528538674 0.005742698907852173
9.260539854949457e-07 0.014895841479301453
Model input: torch.Size([1, 17])
Model output: torch.Size([1, 1])


# Model testing

In [10]:
print("Model Evaluation Starts here !!!")

y_pred_list = []
with torch.no_grad():
     model.eval()
     for X_batch, _ in test_loader:
         X_batch = X_batch.to(device)
         y_test_pred = model(X_batch)
         y_pred_list.append(y_test_pred.cpu().numpy())
y_pred_list = [a.squeeze().tolist() for a in y_pred_list]

# Calculate MAE between predicted and target values in test set

mae_DNN = mean_absolute_error(y_pred_list, y_test)*100
print(mae_DNN)

Model Evaluation Starts here !!!
11.141806919574737
